PAC. Pràctica 2. Tipologia i cicle de vida de les dades. Universitat Oberta de Catalunya.

J. de Curtò i DíAz & I. de Zarzà i Cubero. decurto@uoc.edu dezarza@uoc.edu

Compute statistics from synthetic samples generated by Stylegan2-ada (DrCyZ) and images from NASA Perseverance (CyZ).

Adapt folder paths accordingly. Data is available at:

CyZ: MARS Space Exploration Dataset.

https://github.com/decurtoidiaz/cyz

DrCyZ: Techniques for analyzing and extracting useful information from CyZ.

https://github.com/decurtoidiaz/drcyz

c@decurto.be z@dezarza.be

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/dr_cyz.zip'

Streaming output truncated to the last 5000 lines.
  inflating: c/ZL5_0139_0679280748_535EBY_N0051812ZCAM03183_1100LMJ01_320.png  
  inflating: c/ZR0_0215_0686026759_928EBY_N0072050ZCAM08256_1100LMJ02_320.png  
  inflating: c/ZL0_0237_0687978032_223EBY_N0072050ZCAM08260_1100LMJ01_320.png  
  inflating: c/ZL4_0086_0674575983_428EBY_N0040000ZCAM03137_1100LUJ01_320.png  
  inflating: c/ZR0_0114_0677061433_831EBY_N0041062ZCAM08094_0630LMJ01_320.png  
  inflating: c/ZR0_0215_0686026696_928EBY_N0072050ZCAM08256_1100LMJ02_320.png  
  inflating: c/ZR0_0119_0677500628_303EBY_N0041250ZCAM08113_1100LMJ01_320.png  
  inflating: c/ZL0_0178_0682735766_223EBY_N0062250ZCAM08192_1100LMJ01_320.png  
  inflating: c/ZL0_0237_0687977581_733EBY_N0072050ZCAM03241_1100LMJ01_320.png  
  inflating: c/ZL0_0128_0678297475_693EBY_N0041860ZCAM08128_1100LMJ01_320.png  
  inflating: c/ZR3_0055_0671820910_957EBY_N0032046ZCAM03013_0480LMJ01_320.png  
  inflating: c/ZR0_0241_0688336755_331EBY_N0072440ZCAM08266_1100LMJ03

In [13]:
import cv2
import os

data = []
data2 = []
folder = '/content/c/'
folder2 = '/content/drive/MyDrive/StyleGAN2-ADA/out/dr_cyz_256_1000/'

for filename in os.listdir(folder):
    image = cv2.imread(os.path.join(folder,filename))
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (64,64))
        image = image.flatten()
        data.append([image, folder + filename])

for filename2 in os.listdir(folder2):
    image2 = cv2.imread(os.path.join(folder2,filename2))
    if image2 is not None:
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        image2 = cv2.resize(image2, (64,64))
        image2 = image2.flatten()
        data2.append([image2, folder2 + filename2])

In [5]:
print(len(data))
features, images  = zip(*data)

print(len(data2))
features2, images2  = zip(*data2)

5025
1000


In [7]:
# We propose to compute mean of pixel intensity for statistics
mean_features = []
for c in features:
    mean_features.append(c.mean())

mean_features2 = []
for c2 in features2:
    mean_features2.append(c2.mean())

print(mean_features)
print(mean_features2)


[136.9560546875, 153.189208984375, 132.62744140625, 109.865478515625, 151.930908203125, 147.08154296875, 158.233642578125, 175.22705078125, 135.7158203125, 140.535888671875, 110.69970703125, 162.920654296875, 91.49609375, 189.22509765625, 176.995849609375, 161.188720703125, 184.018310546875, 220.55517578125, 135.678466796875, 201.8916015625, 154.53857421875, 149.31103515625, 187.3193359375, 124.645751953125, 165.731689453125, 168.593505859375, 214.68896484375, 177.197998046875, 133.8212890625, 172.74755859375, 168.664794921875, 174.90478515625, 181.548095703125, 164.535400390625, 152.330322265625, 151.350830078125, 160.36669921875, 126.0322265625, 149.57568359375, 203.15869140625, 152.347412109375, 136.4326171875, 150.3125, 136.90576171875, 143.68017578125, 158.052490234375, 145.3193359375, 185.959716796875, 183.537353515625, 157.303955078125, 193.955810546875, 189.44921875, 124.437255859375, 170.900390625, 199.81982421875, 167.988525390625, 154.255126953125, 162.706787109375, 179.5183

In [10]:
# Test homogeneity of variance
from scipy import stats

print(scipy.stats.levene(mean_features, mean_features2)) #Levene
print(scipy.stats.levene(mean_features, mean_features2, center='mean')) #Brown-Forsyth
print(scipy.stats.bartlett(mean_features, mean_features2)) #Bartlett

LeveneResult(statistic=156.12869754877337, pvalue=2.1729631188690332e-35)
LeveneResult(statistic=166.13819743144077, pvalue=1.611279115945387e-37)
BartlettResult(statistic=874.8317447750159, pvalue=2.905247248253299e-192)


In [11]:
# Test normality
print(scipy.stats.normaltest(mean_features)) #Normality of original samples
print(scipy.stats.normaltest(mean_features2)) #Normality of generated samples

NormaltestResult(statistic=2571.783381290372, pvalue=0.0)
NormaltestResult(statistic=0.11406195050626958, pvalue=0.9445648107792636)


In [12]:
#Test goodness of fit (non-parametric)
print(scipy.stats.kruskal(mean_features,mean_features2)) #Kruskal-Wallis

KruskalResult(statistic=30.97421391554919, pvalue=2.6147941787875364e-08)
